<a href="https://colab.research.google.com/github/parimalakettymuthu/MachineLearning-Projects/blob/main/stackExchange_NN_Multilabelclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Added code for autoreload
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: ignored

In [ ]:
import sys    #Access the google drive, basepath, install required packages and setted path for custom-functions
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')

  !pip install torchtext --upgrade --q
  !pip install torchmetrics --q
  !pip install -quiet torch-lr-finder --q
  !pip install wandb --q --upgrade

  basepath = '/content/drive/My Drive/NLP' 
  sys.path.append('/content/drive/My Drive/NLP/custom-functions')

In [ ]:
#Importing the required packages 
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchmetrics
from torchmetrics.classification import MultilabelF1Score, MultilabelHammingDistance
from torchmetrics.functional.classification import multilabel_f1_score, multilabel_hamming_distance

import joblib
import ast
import wandb

from types import SimpleNamespace
from functools import partial
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer as mlb


In [ ]:
#Defined the google drive folders for accessing/saving models related files
embeddings_folder = Path(basepath)/ 'assignment7/WordEmbeddings'
data_folder = Path(basepath)/ 'assignment7/MultiLabel_Classification'
model_saving_folder = Path(basepath)/ 'assignment7/MultiLabel_Classification'

In [ ]:
data_cleaned = data_folder/ "df_multilabel_hw_cleaned.joblib"   #Access the joblib file and load using the command
stackExchange_dataset = joblib.load(data_cleaned)

In [ ]:
X = stackExchange_dataset['cleaned_text'].values  #Access the X & y value from the imported dataset
y = stackExchange_dataset['Tag_Number'].values

In [ ]:
!pip install swifter -qq  #Installed the swifter package

In [ ]:
import swifter   #Imported the required libraries and convert the Tag_Number field using the given function
import ast
stackExchange_dataset['Tag_Number_list'] = stackExchange_dataset['Tag_Number'].swifter.apply(lambda x: ast.literal_eval(x))

In [ ]:
y_final = stackExchange_dataset['Tag_Number_list'].values  

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer as mlb    #Imported module for creating multi-label-binaries
y_stackExchange_encoding = mlb().fit_transform(y_final)

In [ ]:
from sklearn.model_selection import train_test_split   #Performed the datasplit and created train, test & valid dataset
X_sExchange_train, X_valid_test, y_sExchange_train, y_valid_test = train_test_split(X, y_stackExchange_encoding, test_size=0.4, random_state=42)
X_sExchange_valid, X_sExchange_test, y_sExchange_valid, y_sExchange_test = train_test_split(X_valid_test, y_valid_test, test_size=0.5, random_state=42)

In [ ]:
from gensim.models import KeyedVectors #
pretrained_sExchange_file = str(embeddings_folder/ "model_stackExchange_CBOW.bin")
sExchange_vectors = KeyedVectors.load(pretrained_sExchange_file)

In [ ]:
!pip install torch_lr_finder -qq  #Installed the torch lr finder module

In [ ]:
from torch_lr_finder import LRFinder  #Imported required custom & other module
from Trainer_v4 import Trainer
from data_preparation_HW7 import * 

In [ ]:
#from ff_sequential_model import MLPCustom
from ff_sequential_model_v1 import MLPCustom

In [ ]:
#Creating training dataset with subset 
import random
Trainer.set_seed()
sExchange_trainset = CustomDataset(X_sExchange_train, y_sExchange_train)
se_train_subset_indices = random.sample(range(0, len(sExchange_trainset)), 1000)
se_train_subset = torch.utils.data.Subset(sExchange_trainset, se_train_subset_indices)
se_vocab = get_vocab(sExchange_trainset, min_freq=2)

In [ ]:
type(se_vocab), se_vocab['debug']    #Displayed the type of vocab generated and index of string "debug"

In [ ]:
pretrained_sExchange_file = str(embeddings_folder/ "model_stackExchange_CBOW.bin")  #Imported the pre-traine vectors
sExchange_vectors = KeyedVectors.load(pretrained_sExchange_file)

In [ ]:
pretrained_weights, words_found, words_not_found = get_pretrained_weights(
    vocab = se_vocab,
    pretrained_vectors = sExchange_vectors,
    embedding_dim = 300, #updated from 300 to 150
)

Initialized the required HyperParameters 

In [ ]:
hyperparameters = SimpleNamespace(
# for model
    EMBED_DIM=300,
    VOCAB_SIZE=len(se_vocab),
    OUTPUT_DIM=10,
    HIDDEN_SIZES_LIST=[],
    DPROB_LIST=[],
    NON_LINEARITY=nn.SELU(),
    BATCH_NORM=False,
    
    # for optimizer
    OPTIMIZER="AdamW",
    MOMENTUM=0,
    NESTEROV=False,
    
    # for training
    INITIALIZATION="default",
    EPOCHS=20,
    BATCH_SIZE=100,
    LEARNING_RATE=0.001,
    DATASET="STACKEXCHANGE",
    ARCHITECTURE="emdbag_linear",
    
    # Schedulers
    SCHEDULER="None",

    #gradient clipping
    CLIP_TYPE='norm',
    CLIP_VALUE=2
)

Specifying run name & folder

In [ ]:
#Defined the path for saving model and run name
project_name = "StackExchange NN architecture"
run_name="Task3a experiment18 version11"
run_folder = model_saving_folder / run_name
run_folder.mkdir(exist_ok=True)
log_frequency = 5

Modifying Hyperparameters for Run

In [ ]:
########EXECUTED SEVERAL EXPERIMENTS USING DIFFERENT COMBINATIONS ######

# # run 1 - based on default initialization     
# # Notes: Final Learning rate was set to 1

# # run 2
hyperparameters.HIDDEN_SIZES_LIST = [200]
hyperparameters.DPROB_LIST = [0]
hyperparameters.LEARNING_RATE = 0.001  # reset initial learning rate
# # Notes : Final Learning rate was set to 1.18

# # run 3
#hyperparameters.OPTIMIZER = "Adam"
hyperparameters.LEARNING_RATE = 0.001  # reset initial learning rate
# # Notes : Final Learning rate was set to 0.006

# # run 4
hyperparameters.HIDDEN_SIZES_LIST = [200] + [200]
hyperparameters.DPROB_LIST = [0] + [0]
hyperparameters.LEARNING_RATE = 0.001  # reset initial learning rate
# # Notes : Final Learning rate was set to 0.003

# # run 5
hyperparameters.INITIALIZATION = 'kaiming'
hyperparameters.NON_LINEARITY = nn.ReLU()
hyperparameters.LEARNING_RATE = 0.001 
# # Notes : Final Learning rate was set to 0.003

# # run 6
hyperparameters.HIDDEN_SIZES_LIST = [400] 
hyperparameters.DPROB_LIST = [0] 
hyperparameters.LEARNING_RATE = 0.001  
# # Notes : Final Learning rate was set to 0.005

##run 7
hyperparameters.HIDDEN_SIZES_LIST = [100] + [200] +[200]
hyperparameters.DPROB_LIST = [0] + [0] +[0]
hyperparameters.LEARNING_RATE = 0.001  
# # Notes : Final Learning rate was set to 0.016

#run 8 
hyperparameters.HIDDEN_SIZES_LIST = [400] + [300] +[200] 
hyperparameters.DPROB_LIST = [0] + [0] +[0]
hyperparameters.LEARNING_RATE = 0.001
# Notes : Final Learning rate was set to 1

#run9
hyperparameters.HIDDEN_SIZES_LIST = [400] + [300] +[200]
hyperparameters.DPROB_LIST = [0.5] + [0.5] +[0.5]
hyperparameters.LEARNING_RATE = 0.001
# Notes : Final Learning rate was set to 2

#run10
hyperparameters.HIDDEN_SIZES_LIST = [500] + [400] +[300]
hyperparameters.DPROB_LIST = [0] + [0] +[0]
hyperparameters.LEARNING_RATE = 0.001
# Notes : Final Learning rate was set to 0.9 

#run 11
hyperparameters.HIDDEN_SIZES_LIST = [600] + [500] +[400]
hyperparameters.DPROB_LIST = [0] + [0] +[0]
hyperparameters.LEARNING_RATE = 0.001
# Notes : Final Learning rate was set to 0.1

#run 12
hyperparameters.HIDDEN_SIZES_LIST = [600] + [600] +[400]
hyperparameters.DPROB_LIST = [0] + [0] +[0]
hyperparameters.LEARNING_RATE = 0.001
# Notes : Final Learning rate was set to 0.5 

#run13
hyperparameters.HIDDEN_SIZES_LIST = [500] + [400] +[300]
hyperparameters.DPROB_LIST = [0] + [0] +[0]
hyperparameters.LEARNING_RATE = 0.001
# Notes : Final Learning rate was set to 0.1

#run14
hyperparameters.HIDDEN_SIZES_LIST = [600] + [500] +[400]
hyperparameters.DPROB_LIST = [0.2] + [0.1] +[0.1]
hyperparameters.LEARNING_RATE = 0.001
# Notes : Final Learning rate was set to 0.1

#run14
hyperparameters.HIDDEN_SIZES_LIST = [600] + [500] +[400]
hyperparameters.DPROB_LIST = [0.2] + [0.2] +[0.2]
hyperparameters.LEARNING_RATE = 0.001
# # Notes : Final Learning rate was set to 0.1

#run15 
hyperparameters.DPROB_LIST = [0.2] + [0.2] +[0.1]
CLIP_TYPE='norm',
CLIP_VALUE=2,
EPOCHS=10,
hyperparameters.WEIGHT_DECAY = 0.1
# # Notes : Final Learning rate was set to 0.1

#run 16
#One cycle scheduler
hyperparameters.LEARNING_RATE = 0.001   
hyperparameters.WEIGHT_DECAY = 10
# hyperparameters.SCHEDULER='OneCyclicLR'
# hyperparameters.SCHEDULER_MAX_LR=0.01
# hyperparameters.SCHEDULER_DIV_FACTOR=25
# hyperparameters.SCHEDULER_FINAL_DIV_FACTOR=1e3
hyperparameters.EPOCHS = 10
# # Notes : Final Learning rate was set to 0.1

#run17
hyperparameters.LEARNING_RATE = 0.001 
hyperparameters.HIDDEN_SIZES_LIST = [200]+ [200] 
CLIP_VALUE=10
hyperparameters.WEIGHT_DECAY = 5
# # Notes : Final Learning rate was set to 0.5

#run18
hyperparameters.LEARNING_RATE = 0.001 
hyperparameters.HIDDEN_SIZES_LIST = [100]+ [200]+[200]
#hyperparameters.DPROB_LIST = [0] + [0] +[0]
CLIP_VALUE=2
hyperparameters.EPOCHS = 20
hyperparameters.WEIGHT_DECAY = 0
# # Notes : Final Learning rate was set to 0.1

Configuring the trainer 

In [ ]:
#Invoked the set_seed from Trainer class and created the partial function using generated vocabulary
Trainer.set_seed()
se_collate_fn = partial(collate_batch, vocab=se_vocab)

In [ ]:
#Created Data Loader, initialized the loss function, model, optimizer & device
se_train_loader, _ = get_loaders(trainset=sExchange_trainset, validset=None, 
                                         batch_size_=hyperparameters.BATCH_SIZE,
                                         collate_fn=se_collate_fn)

se_loss_function = nn.BCEWithLogitsLoss()
stackExchange_model = MLPCustom(hyperparameters.EMBED_DIM,
                                hyperparameters.VOCAB_SIZE,
                                hyperparameters.HIDDEN_SIZES_LIST,
                                hyperparameters.DPROB_LIST, 
                                hyperparameters.OUTPUT_DIM, 
                                hyperparameters.NON_LINEARITY,
                                hyperparameters.BATCH_NORM,))
def init_weights(m):
  if type(m)=="nn.Linear":
    torch.nn.init.kaiming_normal_(m.weight)
    torch.nn.init.zeros_(nn.bias)

#Apply initialization to all modules
if hyperparameters.INITIALIZATION == 'kaiming':
  stackExchange_model.apply(init_weights)


#Defining optimizer
def get_optimizer():
  if hyperparameters.OPTIMIZER == 'SGD':
    optimizer = torch.optim.SGD(
        stackExchange_model.parameters(),
        lr = hyperparameters.LEARNING_RATE,
        momentum = hyperparameters.MOMENTUM,
        nesterov = hyperparameters.NESTEROV,
        weight_decay = hyperparameters.WEIGHT_DECAY
    )
  else:
    optimizer = torch.optim.AdamW(
        stackExchange_model.parameters(),
        lr = hyperparameters.LEARNING_RATE,
        weight_decay = hyperparameters.WEIGHT_DECAY
    )
  return optimizer

sExchange_optimizer = get_optimizer()
sExchange_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
sExchange_optimizer   #Displayed the initialized optimizer

In [ ]:
stackExchange_model   #Displayed the initialized model

Trainer

In [ ]:
#Created the Trainer object & invoked several set functions
sEXchange_trainer = Trainer(
    model=stackExchange_model, optimizer=sExchange_optimizer, 
    criterion=se_loss_function,
    device=sExchange_device
)
sEXchange_trainer.set_optimizer(get_optimizer())
sEXchange_trainer.set_loaders(se_train_loader)
sEXchange_trainer.set_gradient_clipping(hyperparameters.CLIP_TYPE, hyperparameters.CLIP_VALUE, norm_type=2)
steps_per_epoch = len(sEXchange_trainer.train_loader)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(sEXchange_trainer.optimizer, 
#                                                 hyperparameters.SCHEDULER_MAX_LR, 
#                                                 steps_per_epoch=steps_per_epoch,
#                                                 epochs=hyperparameters.EPOCHS,
#                                                 div_factor=hyperparameters.SCHEDULER_DIV_FACTOR,
#                                                 final_div_factor=hyperparameters.SCHEDULER_FINAL_DIV_FACTOR)
# sEXchange_trainer.set_lr_scheduler(scheduler=scheduler)

Learning Rate Finder

In [ ]:
#Called the learning rate finder function to suggest LR based on the hyperparameters defined on the model
sEXchange_trainer.lr_finder_range_test(se_train_loader)

In [ ]:
hyperparameters.LEARNING_RATE = 0.1   #Based on suggestion, initialized learning rate

Set trainer based on hyperparameters

In [ ]:
#Resetting optimizer
sEXchange_trainer.set_optimizer(get_optimizer())

#Setting metric for multilabel classification
se_train_metric = MultilabelHammingDistance(num_labels=10)
sEXchange_trainer.set_metric(se_train_metric.to(sExchange_device))

In [ ]:
#Setting wandb 
sEXchange_trainer.set_wandb(
   project_name = project_name,
   run_name = run_name,
   config = hyperparameters,
   log_batch=True, 
   log_frequency=log_frequency
 )

In [ ]:
sEXchange_trainer.learning_rates

Sanity Check

In [ ]:
#Performing the sanity check
sEXchange_trainer.sanity_check(num_classes=2)   

In [ ]:
#Training the model
sEXchange_trainer.train(num_epochs=hyperparameters.EPOCHS, multilabel=True)

In [ ]:
#Plotting the graph based on model training
sEXchange_trainer.plot_history()

Finish Run

In [ ]:
wandb.finish()  #Displaying the cumulative results using wandb function